In [1]:
import sys
import struct
import os
from scapy.all import *
from matplotlib import pyplot as plt
from prettytable import PrettyTable

filename = "zoom/zoom_full.pcapng"
a = rdpcap(filename)
sessions = a.sessions()

In [2]:
TCP_cons = []
UDP_cons = []


def match_TCP(con, pkt):
    if pkt[IP].src == con['s_ip'] and \
       pkt[IP].dst == con['d_ip'] and \
       pkt[TCP].sport == con['s_port'] and \
       pkt[TCP].dport == con['d_port']:
        return True
    else:
        return False
    
def match_UDP(con, pkt):
    if pkt[IP].src == con['s_ip'] and \
       pkt[IP].dst == con['d_ip'] and \
       pkt[UDP].sport == con['s_port'] and \
       pkt[UDP].dport == con['d_port']:
        return True
    else:
        return False
        

for session in sessions:
    for idx,packet in enumerate(sessions[session]):
        if(packet.haslayer(ARP) or packet.haslayer(DNS)):
            continue
             
        if(packet.haslayer(IP)):
            if(packet.haslayer(TCP)):                
                flag = False
                for i,con in enumerate(TCP_cons):
                    if match_TCP(con, packet):
                        TCP_cons[i]['packets'].append({
                            "seq":packet[TCP].seq, "len":len(packet[TCP].payload),
                            "flag":packet[TCP].flags, "order": idx
                        })
                        flag = True
                        break
                if not flag:
                    TCP_cons.append({
                        's_ip':packet[IP].src,'d_ip':packet[IP].dst,
                        's_port':packet[TCP].sport,'d_port':packet[TCP].dport,
                        'type':"TCP",
                        'packets':[{
                            "seq":packet[TCP].seq, "len":len(packet[TCP].payload),
                            "flag":packet[TCP].flags, "order": idx
                        }]
                    })
                pass
        
            elif(packet.haslayer(UDP)):
                flag = False
                for i,con in enumerate(UDP_cons):
                    if match_UDP(con, packet):
                        UDP_cons[i]['packets'].append({
                            "len":len(packet[UDP].payload),
                            "order": idx
                        })
                        flag = True
                        break
                if not flag:
                    UDP_cons.append({
                        's_ip':packet[IP].src,'d_ip':packet[IP].dst,
                        's_port':packet[UDP].sport,'d_port':packet[UDP].dport,
                        'type':"UDP",
                        'packets':[{
                            "len":len(packet[UDP].payload),
                            "order": idx
                        }]
                    })
                pass
                pass
            else:
                pass
#                 break

            

In [3]:
len(TCP_cons)

16

In [4]:
TCP_cons[0]

{'s_ip': '172.16.105.110',
 'd_ip': '170.114.10.89',
 's_port': 60450,
 'd_port': 443,
 'type': 'TCP',
 'packets': [{'seq': 1289558202,
   'len': 31,
   'flag': <Flag 24 (PA)>,
   'order': 4},
  {'seq': 1289558233, 'len': 0, 'flag': <Flag 20 (RA)>, 'order': 5}]}

In [5]:
len(UDP_cons)

6

In [6]:
all_cons = TCP_cons + UDP_cons
all_cons.sort(key=lambda con: con['packets'][0]['order'])

In [7]:
x = PrettyTable()

x.field_names = ["Connection", "Type", "Num Packet", "Total Data"]

for con in all_cons:
    x.add_row([f"{con['s_ip']}:{con['s_port']}->{con['d_ip']}:{con['d_port']}", 
               con['type'], 
               len(con['packets']), 
               sum([pkt['len'] for pkt in con['packets']])])
    

print(x)

+-------------------------------------------+------+------------+------------+
|                 Connection                | Type | Num Packet | Total Data |
+-------------------------------------------+------+------------+------------+
|  172.16.105.110:60450->170.114.10.89:443  | TCP  |     2      |     31     |
|  172.16.105.110:60452->170.114.10.89:443  | TCP  |     2      |     31     |
|  172.16.105.110:60454->170.114.10.89:443  | TCP  |     2      |     31     |
|  172.16.105.110:56786->170.114.10.83:443  | TCP  |     2      |     31     |
|  172.16.105.110:52210->170.114.10.71:443  | TCP  |     11     |    3015    |
|  170.114.10.71:443->172.16.105.110:52210  | TCP  |     9      |   34949    |
| 172.16.105.110:45782->198.251.148.137:443 | TCP  |     8      |    517     |
| 172.16.105.110:60832->198.251.134.174:443 | TCP  |     13     |    1874    |
| 172.16.105.110:32936->198.251.148.138:443 | TCP  |     8      |    517     |
| 172.16.105.110:40088->198.251.135.168:443 | TCP  |